In [9]:
import torch
import numpy as np
import pandas as pd
import re
import torch
import sklearn
import nltk
import joblib
import pandas as pd

from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import set_seed

import clickhouse_connect

from sentence_transformers import SentenceTransformer

from sklearn.neighbors import NearestNeighbors

from utils.utils import Chunker, Generator, Retriever
from utils.llm_config import GenerationConfig

device = 'cuda'

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
client = clickhouse_connect.get_client(host='62.84.115.43', port=8123, username='default', password='g5oHPYX4', database='dev')

In [3]:
retriever_model = SentenceTransformer("BAAI/bge-m3", device='cuda')
retriever_model.max_seq_length = 512

generator_model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).cuda().eval()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
generator_config = GenerationConfig()
generator = Generator(generator_tokenizer, generator_model, config=generator_config)

In [7]:
queries = pd.read_csv('Book1 (1).csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

In [12]:
queries

,site_id,question
0,https://cbr.ru/Crosscut/LawActs/File/6620/,Кто может принимать участие в закупках Банка Р...
1,https://cbr.ru/Queries/UniDbQuery/File/90134/2...,Cроки предоставления информации об активах и о...
2,https://cbr.ru/Queries/UniDbQuery/File/90002/12/,Что считается уровнем риска?
3,https://cbr.ru/Queries/UniDbQuery/File/90002/12/,Какими видами рисков обязательно должен управл...
4,https://cbr.ru/Crosscut/LawActs/File/6144/,На какие банковские операции распространяется ...
5,http://pravo.gov.ru/proxy/ips/?docbody=&prevDo...,Регионы проведения эксперимента по партнерском...
6,http://pravo.gov.ru/proxy/ips/?docbody=&nd=102...,Что такое ВЕБ.РФ?
7,http://pravo.gov.ru/proxy/ips/?docbody=&nd=102...,Какие полномочия у председателя ВЭБ.РФ?
8,https://cbr.ru/Queries/UniDbQuery/File/90002/49/,Какие условия совершения операций по выдаче ми...


## Chunker

In [ ]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)

texts = client.query_df('select * from document').set_index('url')
chunks, urls = chunker.split_texts(texts)
client.insert(table='chunk', data=[chunks, urls], column_names=['text', 'url'], column_oriented=True)
chunks_df = client.query_df('select * from chunk').set_index('uuid')
embeddings = retriever_model.encode(chunks_df.text, batch_size=32, normalize_embeddings=True, show_progress_bar=True)
client.insert(table='chunk_embedding', data=[chunks_df.index.tolist(), embeddings.tolist()], column_names=['chunk_uuid', 'embedding'], column_oriented=True)

## Retriever & Generator

In [10]:
retriever = Retriever(retriever_model, client)

In [28]:
question = queries.question.sample(1).item()

In [31]:
question

'Cроки предоставления информации об активах и обязательствах резидентом-экспортером'

In [32]:
print(generator.get_answer(question, *retriever.get_neighbors(question), temperature=0.8))

В срок не позднее пятнадцати рабочих дней после дня окончания календарного месяца.

Использованные документы:
1) https://cbr.ru/Queries/UniDbQuery/File/90134/2518/ 
2) http://pravo.gov.ru/proxy/ips/?docbody=&nd=102084553 

